# SVG Images and Cairo

[Cairo](https://www.cairographics.org/) is a widely used graphics library for
working with SVG (Scalable Vector Graphics) images. Cairo is written in C but
there are several Python libraries available to make it accessible to py5.

Converting SVG Images to Py5Image objects with Cairo and
[](/reference/sketch_convert_shape) is useful to py5 users that want to use SVG
content in a Sketch that uses a rasterized renderer (such as the default JAVA2D
renderer or the OpenGL renderers P2D or P3D).

Processing can also load SVG images as Py5Shape objects with
[](/reference/sketch_load_shape) but the method supports only a small subset of
the full SVG specification. The method is not intended to be a comprehensive SVG
interpreter, as providing that functionality would be a significant undertaking.
Nevertheless, the method's limitations will result in problems when loading SVG
files created in sophisticated SVG editors such as Inkscape or Adobe Illustrator.

## Setup

First you must install the 2D graphics library Cairo. The easiest way to do this
is with conda using the conda-forge channel. This will work for any operating
system.

```bash
conda install cairo --channel conda-forge
```

If you don't want to use conda, refer to the install instructions in the [Cairo
documentation](https://www.cairographics.org/download/). Installing Cairo on
Windows without conda is challenging.

Next you will need to install the Python libraries that py5 uses to access Cairo
and work with SVG files. The cairosvg library depends on cairocffi.

```bash
conda install --channel conda-forge cairosvg cairocffi
```

You can also install these with `pip`.

TODO: remove this from the install page

## Convert SVG Images

Let's start by importing the necessary libraries for this demonstration.

In [ ]:
from IPython.display import SVG

import py5_tools
import py5

Here is the SVG file we will be working with. Observe the SVG image uses a
color gradient and has centered text.

In [ ]:
with open('images/py5_is_awesome.svg') as f:
    svg_code = f.read()

SVG(svg_code)

Now let's use that in our Sketch. Notice we are passing the SVG image path to
[](/reference/convert_image). It will read the SVG file, and then the cairosvg
library will send it to Cairo for rastorization via cairocffi. The method call
returns a `Py5Image` object.

In [ ]:
def setup():
    py5.size(500, 500)

    svg = py5.convert_image('images/py5_is_awesome.svg')
    assert isinstance(svg, py5.Py5Image)

    py5.image(svg, 0, 0)


py5.run_sketch()

Here's what that looks like:

In [ ]:
py5_tools.screenshot()

In [ ]:
import time

time.sleep(1)

py5.exit_sketch()

If instead we had tried to load the SVG file as a Py5Shape object with
[](/reference/sketch_load_shape), we would have seen this:

In [ ]:
def setup():
    py5.size(500, 500)

    svg = py5.load_shape('images/py5_is_awesome.svg')

    py5.shape(svg, 0, 0)


py5.run_sketch()

In [ ]:
py5_tools.screenshot()

In [ ]:
time.sleep(1)

py5.exit_sketch()

It can't handle the color gradient and the text alignment is wrong. Therefore, using [](/reference/convert_image) is a better option for working with this SVG image.

## Optional Conversion Parameters

The [](/reference/sketch_convert_shape) method provides a few optional customization parameters for SVG files.

The most useful optional parameter is `scale`, allowing you to change the scale of the
converted SVG image. There are some others such as `negate_colors` that will invert
the SVG's color palette.

In [ ]:
def setup():
    py5.size(500, 500)

    svg1 = py5.convert_image('images/py5_is_awesome.svg', scale=0.5)
    svg2 = py5.convert_image('images/py5_is_awesome.svg', scale=0.5, negate_colors=True)

    py5.image(svg1, 0, 0)
    py5.image(svg2, 250, 0)
    py5.image(svg2, 0, 250)
    py5.image(svg1, 250, 250)


py5.run_sketch()

Here's the result:

In [ ]:
py5_tools.screenshot()

In [ ]:
time.sleep(1)

py5.exit_sketch()

The full list of optional parameters is below.

| keyword argument | description |
|---|---|
| parent_width | width of parent container in pixels |
| parent_height | height of parent container in pixels |
| dpi | ratio between 1 inch and 1 pixel |
| scale | output scaling factor |
| unsafe | resolve XML entities and allow very large files |
| background_color | background color to replace transparent background |
| negate_colors | negate SVG colors |
| invert_images | negate colors in embedded images |
| output_width | desired output width in pixels |
| output_height | desired output height in pixels |

## Cairo Surfaces

```bash
conda install --channel conda-forge pycairo
```